In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.geometry import box, Point, Polygon, MultiPolygon
from shapely.wkt import loads

In [2]:
# lendo o arquivo .shp
caminho_shp = '../Estudos/datasets/movi/data/paradas.shp'
df = gpd.read_file(caminho_shp)

In [3]:
# pegando algumas informações
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5457 entries, 0 to 5456
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   parada      5457 non-null   object  
 1   descricao   5456 non-null   object  
 2   situacao    5457 non-null   object  
 3   estrutura_  4948 non-null   object  
 4   tipo        5455 non-null   object  
 5   geometry    5457 non-null   geometry
dtypes: geometry(1), object(5)
memory usage: 255.9+ KB


In [4]:
# O que temos na coluna 'geometry' ?
df['geometry']

0       POINT (203628.051 8238910.255)
1       POINT (203626.569 8238879.548)
2       POINT (203515.071 8238613.416)
3       POINT (203481.047 8238551.784)
4       POINT (203579.703 8239213.734)
                     ...              
5452    POINT (167354.394 8264495.757)
5453    POINT (167311.355 8264500.454)
5454    POINT (163265.979 8265332.396)
5455    POINT (162912.871 8269061.030)
5456    POINT (163217.911 8264159.565)
Name: geometry, Length: 5457, dtype: geometry

In [5]:
# Qual sistemas estamos?
# acessando o atributo crs
df.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
# O folium trabalha no sistema WGS 84(EPSG:4326), o correto é converter para esse formato
df = df.to_crs('EPSG:4326')

In [7]:
# Agora conseguimos plotar

# Pegando uma cordenada pelo indice
x = df.geometry.x.iloc[0] # longitude
y = df.geometry.y.iloc[0] # latidute
print(f'Coordenada x: {x}\nCoordena y: {y}')

Coordenada x: -47.76791017881488
Coordena y: -15.911319542624627


In [8]:
import folium

# Criar um mapa Folium centrado nas coordenadas do ponto selecionado
mapa = folium.Map(location=[y, x], zoom_start=12)

# Adicionar um marcador para o ponto no mapa
folium.Marker(location=[y, x]).add_to(mapa)

# Exibir o mapa
mapa

In [9]:
df['descricao'].iloc[0]

'Avenida Morro da Cruz'

In [10]:
# Lendo o arquivo das linhas
file = '../Estudos/datasets/movi/data/Linhas.shp'
linhas = gpd.read_file(file)
linhas.head(5)

,id,linha,nome,sentido,faixa_tari,tarifa,situacao,bacia,tipo_da_li,situacao_d,geometry
0,14832,0.018,PARANOÁ PARQUE / W3 NORTE (VIA PONTE JK),IDA,METROPOLITANA 2,5.5,Ativa,2,Rodoviária,T,"LINESTRING (201967.451 8255590.195, 201946.837..."
1,14833,0.018,PARANOÁ PARQUE / W3 NORTE (VIA PONTE JK),VOLTA,METROPOLITANA 2,5.5,Ativa,2,Rodoviária,T,"LINESTRING (188410.823 8258088.738, 188394.509..."
2,13249,100.2,Paranoá (Paranoá Parque) / Rodoviária do Plano...,VOLTA,METROPOLITANA 2,5.5,Ativa,2,Rodoviária,T,"LINESTRING (191217.354 8251726.387, 191224.211..."
3,10696,0.946,Vicente Pires / Rodoviária do Plano Piloto (EIXO),IDA,METROPOLITANA 2,5.5,Ativa,5,Rodoviária,T,"LINESTRING (172882.705 8251337.072, 172882.582..."
4,15435,0.373,Samambaia Norte (2 Avenida) / Rodoviária do Pl...,IDA,METROPOLITANA 2,5.5,Ativa,3,Rodoviária,T,"LINESTRING (162573.278 8241365.140, 162605.242..."


In [11]:
# instalando uma nova biblioteca necessária
%pip install shapely



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
linestringRecanto = linhas[linhas['linha']=='0.813']
linestringRecantoIda = linestringRecanto.iloc[1] # ida
linestringRecantoVolta = linestringRecanto.iloc[0] # volta


In [13]:
import folium
from shapely.geometry import LineString

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Criar um mapa inicial
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

# Obter o objeto LineString a partir da coluna "geometry" do DataFrame
linestring1 = linestringRecanto.geometry.iloc[1] # Ida
linestring2 = linestringRecanto.geometry.iloc[0] # Volta

# Converter as coordenadas do Linestring para o formato que o Folium espera (latitude, longitude)
lat_lon_coordsIda = [(lat1, lon1) for lon1, lat1 in linestring1.coords]
lat_lon_coordsVolta = [(lat2, lon2) for lon2, lat2 in linestring2.coords]


# Adicionar o Linestring ao mapa como uma polilinha (linha poligonal)

# Plotando a ida
folium.PolyLine(locations=lat_lon_coordsIda, color='blue', tooltip=linestringRecanto['linha'].iloc[1]+ ' ' + linestringRecanto['sentido'].iloc[1]).add_to(m)

# Plotando a volta
folium.PolyLine(locations=lat_lon_coordsVolta, color='red', tooltip=linestringRecanto['linha'].iloc[0]+ ' ' + linestringRecanto['sentido'].iloc[0]).add_to(m)

# Mostrar o mapa
m.save('mapaLinestring.html')
m

In [14]:
# Verificando paradas proximas a linestring

paradas = gpd.read_file("../Estudos/datasets/movi/data/paradas.shp")
paradas.tail(5)

,parada,descricao,situacao,estrutura_,tipo,geometry
5452,1163,NUCLEO RURAL RODEADOR - LADO OPOSTO ESCOLA ROD...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (167354.394 8264495.757)
5453,1164,NUCLEO RURAL RODEADOR - EM FRENTE ESCOLA RODEA...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (167311.355 8264500.454)
5454,1166,ANTES DA ENTRADA PARA A DF 445 NORTE - KM 7 DF...,ATIVA,SEM ESTRUTURA,Tipo C,POINT (163265.979 8265332.396)
5455,1159,LADO OPOSTO A ENTRADA DA RADIOBRAS E DF 220 DF...,ATIVA,SEM ESTRUTURA,Habitual,POINT (162912.871 8269061.030)
5456,6374,Km 07 - Gleba 02 - Chácara 116 - Chácara Beatr...,ATIVA,SEM ESTRUTURA,Padrão,POINT (163217.911 8264159.565)


In [15]:
# convertendo para o sistema adequado
paradas = paradas.to_crs('EPSG:4326')
paradas

,parada,descricao,situacao,estrutura_,tipo,geometry
0,6710,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76791 -15.91132)
1,6711,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76793 -15.91160)
2,6712,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76900 -15.91399)
3,6713,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76933 -15.91454)
4,6827,Residencial Vitoria,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76832 -15.90857)
...,...,...,...,...,...,...
5452,1163,NUCLEO RURAL RODEADOR - LADO OPOSTO ESCOLA ROD...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-48.10282 -15.67575)
5453,1164,NUCLEO RURAL RODEADOR - EM FRENTE ESCOLA RODEA...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-48.10322 -15.67570)
5454,1166,ANTES DA ENTRADA PARA A DF 445 NORTE - KM 7 DF...,ATIVA,SEM ESTRUTURA,Tipo C,POINT (-48.14080 -15.66765)
5455,1159,LADO OPOSTO A ENTRADA DA RADIOBRAS E DF 220 DF...,ATIVA,SEM ESTRUTURA,Habitual,POINT (-48.14358 -15.63395)


In [76]:
import folium
from shapely.geometry import LineString, Point

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Criar um mapa inicial
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

# Obter o objeto LineString a partir da coluna "geometry" do DataFrame
linestring = linestringRecanto.geometry.iloc[0]

# Iterar sobre os pontos de paradas e verificar se estão contidos na Linestring
for idx, ponto_parada in paradas.iterrows():
    ponto = ponto_parada["geometry"]
    point_obj = Point(ponto.x, ponto.y)
    if linestring.contains(point_obj):
        print(f"O ponto de parada {ponto_parada['descrição']} está contido na Linestring.")


In [78]:
import folium
from shapely.geometry import LineString, Point

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Criar um mapa inicial
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

# Obter o objeto LineString a partir da coluna "geometry" do DataFrame
linestring = linestringRecanto.geometry.iloc[0]

# Função para aproximar as coordenadas para 2 casas decimais
def aproximar_coordenadas(coord):
    return round(coord, 2)

# Iterar sobre os pontos de paradas e verificar se estão contidos na Linestring
for idx, ponto_parada in paradas.iterrows():
    ponto = ponto_parada["geometry"]
    ponto_aproximado = Point(aproximar_coordenadas(ponto.x), aproximar_coordenadas(ponto.y))
    linestring_aproximada = LineString([Point(aproximar_coordenadas(x), aproximar_coordenadas(y)) for x, y in linestring.coords])
    if linestring_aproximada.contains(ponto_aproximado):
        print(f"O ponto de parada {ponto_parada['descricao']} está contido na Linestring.")


O ponto de parada EQN 513/514 W3 NORTE está contido na Linestring.
O ponto de parada SQN 113 EIXO W NORTE está contido na Linestring.
O ponto de parada ENTRADA PARA Q. 508 CNJUNTOS 2. 3 E 3 AV. MONJOLO está contido na Linestring.
O ponto de parada LADO OPOSTO Q. 508 APOS ENTRADA DA RUA DO PARQUE AV. MONJOLO está contido na Linestring.
O ponto de parada QUADRA 308 ANTES DA ENTRADA QUADRA 309 AV. MONJOLO está contido na Linestring.
O ponto de parada QUADRA 111 - AO LADO DO CENTRO DE ENSINO AV. REC. EMAS está contido na Linestring.
O ponto de parada QUADRA 112  ANTES QUADRA POLIESPORTIVA AV. REC. EMAS está contido na Linestring.
O ponto de parada QUADRA 113 - FRENTE DO CEF 113 AV. REC. EMAS está contido na Linestring.
O ponto de parada QUADRA 114 - LOTE 06 AV. REC. EMAS está contido na Linestring.
O ponto de parada QUADRA 114 - APOS O BALAO AV. REC. EMAS está contido na Linestring.
O ponto de parada QUADRA 309 AV. REC. EMAS está contido na Linestring.
O ponto de parada QUADRA 308 - APOS O

In [80]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Lista para armazenar as paradas encontradas
paradas_encontradas = []

# Função para aproximar as coordenadas para 2 casas decimais
def aproximar_coordenadas(coord):
    return round(coord, 2)

# Iterar sobre os pontos de paradas e verificar se estão contidos na Linestring
for idx, ponto_parada in paradas.iterrows():
    ponto = ponto_parada["geometry"]
    ponto_aproximado = Point(aproximar_coordenadas(ponto.x), aproximar_coordenadas(ponto.y))
    linestring_aproximada = LineString([Point(aproximar_coordenadas(x), aproximar_coordenadas(y)) for x, y in linestring.coords])
    if linestring_aproximada.contains(ponto_aproximado):
        paradas_encontradas.append(ponto_parada)

# Criar um novo GeoDataFrame com as paradas encontradas
gdf_paradas_encontradas = gpd.GeoDataFrame(paradas_encontradas, crs='EPSG:4326')

# Plotar as paradas de ônibus no mapa com o Folium
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

for idx, ponto_parada in gdf_paradas_encontradas.iterrows():
    lat, lon = ponto_parada["geometry"].y, ponto_parada["geometry"].x
    folium.Marker([lat, lon], popup=ponto_parada['descricao']).add_to(m)

# Mostrar o mapa com as paradas de ônibus
m.save('mapa_paradas_onibus.html')


In [81]:
m